In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import math
import gensim
from gensim.models import Word2Vec 
import time
from sklearn.metrics import confusion_matrix

# Prepare the data

In [2]:
def prepare(df):
    
    y = df.label.tolist()
    X = np.matrix(df.drop(labels = ['label'], axis = 1)).astype(np.float)

    return X, y

# XGB

In [3]:
from xgboost import XGBClassifier
import xgboost as xgb

In [4]:
trainPath = ['/scratch/by8jj/stratified samples/test of test/train/train_11.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_12.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_13.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_14.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_15.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_16.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_17.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_18.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_19.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_20.csv',
             '/scratch/by8jj/stratified samples/test of test/train/train_21.csv',
            ]

In [5]:
testPath = ['/scratch/by8jj/stratified samples/test of test/test/test_12.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_13.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_14.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_15.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_16.csv',
            '/scratch/by8jj/stratified samples/test of test/test/test_17.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_18.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_19.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_20.csv',
            '/scratch/by8jj/stratified samples/test of test/test/test_21.csv',
           '/scratch/by8jj/stratified samples/test of test/test/test_22.csv',
            ]

In [6]:
for train, test in zip(trainPath, testPath):
        
    #drop different port numbers
    df_train = pd.read_csv(train)
    temp1 = set(df_train.columns)
    
    #for test in testPath:
    df_test = pd.read_csv(test)
    temp2 = set(df_test.columns)
    
    df_train = df_train.drop(list(temp1 - temp2), axis = 1)
    df_test = df_test.drop(list(temp2 - temp1), axis = 1)
        
   
    X, y = prepare(df_train)
    
    model = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=150, 
                      reg_alpha = 0.3,
                      max_depth=5,
                      verbosity = 0,
                      n_jobs = 8)
    
    X_A, X_B, Y_A, Y_B  = train_test_split(X, y, test_size=0.33)
    
    eval_set = [(X_A, Y_A), (X_B, Y_B)]
    eval_metric = ["auc","error"]
    model.fit(X_A, Y_A, eval_metric=eval_metric, eval_set=eval_set)
   
    
    X_test, y_test = prepare(df_test)
    y_pred = model.predict_proba(X_test)
    
    
    temp = [1 if x[1]>0.8 else 0 for x in y_pred]
    cm= confusion_matrix(y_test, temp)
    tn, fp, fn, tp = cm.ravel()
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    fpr = fp/(fp+ tn)
    accuracy = (tp + tn)/(tn + tp + fn + fp)
    F1 = 2 * (precision * recall) / (precision + recall)
    print("precision:", precision*100)
    print("recall:", recall*100)
    print("false positive rate:", fpr*100)
    print("accuracy",  accuracy*100)
    print("F1-score", F1)
    
    
    pd.DataFrame(y_pred).to_csv('/scratch/by8jj/stratified samples/test of test/xgb/'+ train[-12:-4] + '-' + test[-11:-4] + '.csv', index = False)


[23:20:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[0]	validation_0-auc:0.925686	validation_0-error:0.096602	validation_1-auc:0.925962	validation_1-error:0.096693
[23:20:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[1]	validation_0-auc:0.934292	validation_0-error:0.081063	validation_1-auc:0.934535	validation_1-error:0.081167
[23:20:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[2]	validation_0-auc:0.938634	validation_0-error:0.095006	validation_1-auc:0.938865	validation_1-error:0.095074
[23:20:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[3]	validation_0-auc:0.9478	validation_0-error:0.095098	validation_1-auc:0.948053	validation_1-error:0.095173
[23:20:08] /workspace/src/tree/updater_prune.cc:74: tree pruni

[23:20:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[35]	validation_0-auc:0.97758	validation_0-error:0.051469	validation_1-auc:0.977749	validation_1-error:0.051478
[23:20:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[36]	validation_0-auc:0.977627	validation_0-error:0.051469	validation_1-auc:0.977794	validation_1-error:0.051496
[23:20:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[37]	validation_0-auc:0.977616	validation_0-error:0.051453	validation_1-auc:0.977789	validation_1-error:0.051456
[23:20:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[38]	validation_0-auc:0.977599	validation_0-error:0.051445	validation_1-auc:0.977757	validation_1-error:0.05147
[23:20:26] /workspace/src/tree/updater_prune.cc:74: tree p

[23:20:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[70]	validation_0-auc:0.978965	validation_0-error:0.051779	validation_1-auc:0.979087	validation_1-error:0.051757
[23:20:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[71]	validation_0-auc:0.979001	validation_0-error:0.051988	validation_1-auc:0.979123	validation_1-error:0.051942
[23:20:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[72]	validation_0-auc:0.979014	validation_0-error:0.052033	validation_1-auc:0.979136	validation_1-error:0.052001
[23:20:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[73]	validation_0-auc:0.979021	validation_0-error:0.052036	validation_1-auc:0.979142	validation_1-error:0.052003
[23:20:45] /workspace/src/tree/updater_prune.cc:74: tree

[23:21:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[105]	validation_0-auc:0.978437	validation_0-error:0.051646	validation_1-auc:0.978598	validation_1-error:0.051645
[23:21:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[106]	validation_0-auc:0.978453	validation_0-error:0.051645	validation_1-auc:0.978615	validation_1-error:0.051646
[23:21:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[107]	validation_0-auc:0.978467	validation_0-error:0.051634	validation_1-auc:0.978631	validation_1-error:0.05163
[23:21:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[108]	validation_0-auc:0.978419	validation_0-error:0.051645	validation_1-auc:0.978586	validation_1-error:0.051632
[23:21:02] /workspace/src/tree/updater_prune.cc:74: t

[23:21:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[140]	validation_0-auc:0.979205	validation_0-error:0.051542	validation_1-auc:0.979396	validation_1-error:0.051518
[23:21:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[141]	validation_0-auc:0.979213	validation_0-error:0.051538	validation_1-auc:0.979404	validation_1-error:0.051517
[23:21:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[142]	validation_0-auc:0.979209	validation_0-error:0.051355	validation_1-auc:0.979402	validation_1-error:0.051324
[23:21:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[143]	validation_0-auc:0.979212	validation_0-error:0.051277	validation_1-auc:0.979405	validation_1-error:0.051247
[23:21:21] /workspace/src/tree/updater_prune.cc:74: 

...

[23:45:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[137]	validation_0-auc:0.986195	validation_0-error:0.055733	validation_1-auc:0.986097	validation_1-error:0.055749
[23:45:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[138]	validation_0-auc:0.98619	validation_0-error:0.055735	validation_1-auc:0.986088	validation_1-error:0.055753
[23:45:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[139]	validation_0-auc:0.986183	validation_0-error:0.055741	validation_1-auc:0.986082	validation_1-error:0.05576
[23:45:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[140]	validation_0-auc:0.986104	validation_0-error:0.055744	validation_1-auc:0.986016	validation_1-error:0.05577
[23:45:55] /workspace/src/tree/updater_prune.cc:74: tre

[23:46:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[22]	validation_0-auc:0.981449	validation_0-error:0.072055	validation_1-auc:0.981249	validation_1-error:0.072601
[23:46:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[23]	validation_0-auc:0.981422	validation_0-error:0.062718	validation_1-auc:0.981222	validation_1-error:0.063339
[23:46:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[24]	validation_0-auc:0.981317	validation_0-error:0.062677	validation_1-auc:0.981132	validation_1-error:0.063296
[23:47:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[25]	validation_0-auc:0.981398	validation_0-error:0.06259	validation_1-auc:0.981206	validation_1-error:0.063226
[23:47:00] /workspace/src/tree/updater_prune.cc:74: tree 

[23:47:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[57]	validation_0-auc:0.982903	validation_0-error:0.061687	validation_1-auc:0.982716	validation_1-error:0.062299
[23:47:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[58]	validation_0-auc:0.982892	validation_0-error:0.058754	validation_1-auc:0.982706	validation_1-error:0.059305
[23:47:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[59]	validation_0-auc:0.982953	validation_0-error:0.058757	validation_1-auc:0.982766	validation_1-error:0.059307
[23:47:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 0 pruned nodes, max_depth=5
[60]	validation_0-auc:0.982993	validation_0-error:0.060661	validation_1-auc:0.98281	validation_1-error:0.061293
[23:47:24] /workspace/src/tree/updater_prune.cc:74: tree 

[23:47:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[92]	validation_0-auc:0.98422	validation_0-error:0.058605	validation_1-auc:0.984005	validation_1-error:0.05915
[23:47:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[93]	validation_0-auc:0.984268	validation_0-error:0.0586	validation_1-auc:0.984054	validation_1-error:0.059144
[23:47:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[94]	validation_0-auc:0.984279	validation_0-error:0.058602	validation_1-auc:0.984065	validation_1-error:0.05915
[23:47:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[95]	validation_0-auc:0.984269	validation_0-error:0.058613	validation_1-auc:0.984057	validation_1-error:0.05916
[23:47:47] /workspace/src/tree/updater_prune.cc:74: tree pruni

[23:48:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[127]	validation_0-auc:0.984286	validation_0-error:0.058616	validation_1-auc:0.984062	validation_1-error:0.059156
[23:48:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[128]	validation_0-auc:0.984273	validation_0-error:0.058612	validation_1-auc:0.984049	validation_1-error:0.059151
[23:48:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[129]	validation_0-auc:0.984268	validation_0-error:0.058604	validation_1-auc:0.984044	validation_1-error:0.059141
[23:48:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 0 pruned nodes, max_depth=5
[130]	validation_0-auc:0.984269	validation_0-error:0.058606	validation_1-auc:0.984047	validation_1-error:0.059144
[23:48:10] /workspace/src/tree/updater_prune.cc:74: 